# SI 618 WN 2018 - Lab 5: Spark DataFrames

## Submission Instructions:
Please turn in this Jupyter notebook file (both .ipynb and .html formats) on Canvas before midnight.

### Name:  Anthony Cozart
### Uniqname: Jacozart
### People you worked with: Anna Lenhart, Lauren Sheridan

## Objectives:
* Know the basics of PySpark SQL manipulation
* Initialize a Spark Session
* Create a Spark DataFrame from a CSV file
* Select subset of data using .filter()
* Sort DataFrame on one ore more columns using .sortBy()
* Group data and calculate aggregate statistics using .groupBy() and .agg()
* Run SQL queries in Spark
* Know how to transform between Spark DataFrame and Pandas DataFrame



 

## Note: If you get stuck, ask for help or try to move onto the next question.

# Exploration of Chocolate Bar Ratings

## Data Description

This dataset is compiled by Brady Brelinski, Founding Member of the Manhattan Chocolate Society. It provides expert ratings for over 1,700 individual chocolate bars, along with information on their regional origin, percentage of cocoa, the variety of chocolate bean used and where the beans were grown. The dataset was obtained from Kaggle: https://www.kaggle.com/rtatman/chocolate-bar-ratings.

## Metadata
Here's a description of the fields available in this dataset.

| Field | Description |
| --- | --- |
| Company (Maker-if known) | Name of the company manufacturing the bar.| 
| Specific Bean Origin or Bar Name | The specific geo-region of origin for the bar.|
| REF | |
| Review Date | Date of publication of the review.|
| Cocoa Percent | Cocoa percentage (darkness) of the chocolate bar being reviewed.|
| Company Location | Manufacturer base country. |
| Rating | Expert rating for the bar.|
| Bean Type | The variety (breed) of bean used, if provided.|
| Broad Bean Origin | The broad geo-region of origin for the bean.|


## Getting started

#### First, let's import the SparkSession class from pyspark.sql library

In [1]:
from pyspark.sql import SparkSession

#### Then, let's initialize a SparkSession

In [2]:
spark = SparkSession.builder.appName('lab5').getOrCreate()

#### Now let's load the chocolate bar data.  You'll be using ```spark.read.csv(...)``` a lot.

Set inferSchema=True to infer the variable types; set header=True if the first line of the csv file is the header; set sep='\t' for tab-separated file.

In [10]:
df = spark.read.csv("flavors_of_cacao2.txt", inferSchema=True, header=True, sep='\t')

#### Print the schema of the DataFrame.

In [11]:
df.printSchema() 

root
 |-- Company (Maker-if known): string (nullable = true)
 |-- Specific Bean Origin or Bar Name: string (nullable = true)
 |-- REF: integer (nullable = true)
 |-- Review Date: integer (nullable = true)
 |-- Cocoa Percent: string (nullable = true)
 |-- Company Location: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Bean Type: string (nullable = true)
 |-- Broad Bean Origin: string (nullable = true)



#### As part of loading data, it's helpful to get an idea of the shape of the data:

In [12]:
print("The number of rows of the dataset is: ", df.count())
print("The number of columns of the dataset is: ", len(df.columns))

The number of rows of the dataset is:  1795
The number of columns of the dataset is:  9


## Update and Remove Columns

To illustrate how to drop a column, let's create a small DataFrame from a list.

In [13]:
temp_df = spark.createDataFrame(
    [('Chris','SI618',98), ('Kai','SI699',85)],['name','course','score']
)
temp_df.show()

+-----+------+-----+
| name|course|score|
+-----+------+-----+
|Chris| SI618|   98|
|  Kai| SI699|   85|
+-----+------+-----+



In [14]:
temp_df.drop("score").show()

+-----+------+
| name|course|
+-----+------+
|Chris| SI618|
|  Kai| SI699|
+-----+------+



### <font color="magenta">Q1: Drop the column "REF" and save it back to df. We will not use this unknown variable for our analysis.

In [15]:
df.show(10)

+------------------------+--------------------------------+----+-----------+-------------+----------------+------+---------+-----------------+
|Company (Maker-if known)|Specific Bean Origin or Bar Name| REF|Review Date|Cocoa Percent|Company Location|Rating|Bean Type|Broad Bean Origin|
+------------------------+--------------------------------+----+-----------+-------------+----------------+------+---------+-----------------+
|                A. Morin|                     Agua Grande|1876|       2016|          63%|          France|  3.75|         |         Sao Tome|
|                A. Morin|                           Kpime|1676|       2015|          70%|          France|  2.75|         |             Togo|
|                A. Morin|                          Atsane|1676|       2015|          70%|          France|   3.0|         |             Togo|
|                A. Morin|                           Akata|1680|       2015|          70%|          France|   3.5|         |             Togo|

In [16]:
df = df.drop("REF")

#### Some column names are too long and can be shortened

In [17]:
df = df.withColumnRenamed('Company\xa0(Maker-if known)', 'Company Name')
df = df.withColumnRenamed('Specific Bean Origin or Bar Name', 'Specific Bean Origin')

#### Some columns have a blank space in their names, so renaming them can save us trouble

### <font color="magenta">Q2: Replace the blank space of column names with underscore.
Hint1: instead of renaming the columns one by one, you can use a for loop.

Hint2: for each column name, do a split and join the resulting list with an underscore. For example, ",".join(["How are you","Ma'am"]) gives you a string "How are you,Ma'am".

In [18]:
for col in df.schema.names:
    words = col.split(" ")
    renamed = '_'.join(words)
    df = df.withColumnRenamed(col, renamed)

In [19]:
df.show(1)

+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|    A. Morin|         Agua Grande|       2016|          63%|          France|  3.75|         |         Sao Tome|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
only showing top 1 row



In [20]:
# check
df.printSchema() 

root
 |-- Company_Name: string (nullable = true)
 |-- Specific_Bean_Origin: string (nullable = true)
 |-- Review_Date: integer (nullable = true)
 |-- Cocoa_Percent: string (nullable = true)
 |-- Company_Location: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Bean_Type: string (nullable = true)
 |-- Broad_Bean_Origin: string (nullable = true)



The following code convert Cocoa Percent from string type to float type.

In [21]:
from pyspark.sql.functions import regexp_extract
df = df.withColumn("Cocoa_Percent", regexp_extract('Cocoa_Percent', r'^[0-9]+', 0).cast('float'))

## Inspect data

#### Showing the first row is easy.

In [22]:
df.show(1)

+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|    A. Morin|         Agua Grande|       2016|         63.0|          France|  3.75|         |         Sao Tome|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
only showing top 1 row



Alternatively, you can use .take(), which returns a list of Row objects.

In [23]:
df.take(1)

[Row(Company_Name='A. Morin', Specific_Bean_Origin='Agua Grande', Review_Date=2016, Cocoa_Percent=63.0, Company_Location='France', Rating=3.75, Bean_Type='\xa0', Broad_Bean_Origin='Sao Tome')]

#### You can sample a subset of data using .sample()
For example, you can sample 10% of the data points without replacement like this

In [24]:
df.sample(False, 0.1).first()

Row(Company_Name='A. Morin', Specific_Bean_Origin='Madagascar', Review_Date=2013, Cocoa_Percent=70.0, Company_Location='France', Rating=3.0, Bean_Type='Criollo', Broad_Bean_Origin='Madagascar')

### <font color="magenta">Q3: Extract the Company Name and Company Location of the first row.

Hint: use.take(), extract the first element of the list, and select the fields using ("A", "B", "C", ...)

In [31]:
df.select('Company_Name','Company_Location').take(1)

[Row(Company_Name='A. Morin', Company_Location='France')]

## Select Columns

Unlike Pandas, Spark's function input can be simply separated by commas, instead of a list

If you want to select a field or multiple fields, simply field names by commas in the function, such as:

In [32]:
df.select("Company_Name" ,"Specific_Bean_Origin", "Rating").show(1)

+------------+--------------------+------+
|Company_Name|Specific_Bean_Origin|Rating|
+------------+--------------------+------+
|    A. Morin|         Agua Grande|  3.75|
+------------+--------------------+------+
only showing top 1 row



Or you can select in a bracket format 

In [33]:
df.select(df["Company_Name"], df["Specific_Bean_Origin"], df["Rating"]).show(1)

+------------+--------------------+------+
|Company_Name|Specific_Bean_Origin|Rating|
+------------+--------------------+------+
|    A. Morin|         Agua Grande|  3.75|
+------------+--------------------+------+
only showing top 1 row



If the field name doesn't contain a space, then you can also use a dot format:

In [34]:
df.select(df.Company_Name, df.Specific_Bean_Origin, df.Rating).show(1)

+------------+--------------------+------+
|Company_Name|Specific_Bean_Origin|Rating|
+------------+--------------------+------+
|    A. Morin|         Agua Grande|  3.75|
+------------+--------------------+------+
only showing top 1 row



### <font color="magenta">Q4: Take a random sample of 5 records and show only their company names and company locations.

In [95]:
# df.sample(False, 1/(df.count()/5)).
take5 = spark.createDataFrame(df.rdd.takeSample(False,5))
take5.select(take5['Company_Name'], take5['Specific_Bean_Origin']).show()

+--------------------+--------------------+
|        Company_Name|Specific_Bean_Origin|
+--------------------+--------------------+
| Artisan du Chocolat|          Costa Rica|
|           Dandelion|Zorzal Reserva, 2...|
|            Ambrosia|                Peru|
|         Bahen & Co.|          Houseblend|
|Heirloom Cacao Pr...|Maunawili, O'ahu,...|
+--------------------+--------------------+



### Select Columns with WHEN manipulation

PySpark's WHEN corresponds to SQL's CASE WHEN clause

If chocolate bars with smaller than 35% cocoa are considered sweet and otherwise bitter, then we can show this as below:

In [45]:
from pyspark.sql import functions as F
df.select("Company_Name", 
          "Specific_Bean_Origin",
          F.when(df.Cocoa_Percent<35, "Sweet").otherwise("Bitter").alias("SweetOrBitter"), 
          "Rating", "Review_Date").show(5)

+------------+--------------------+-------------+------+-----------+
|Company_Name|Specific_Bean_Origin|SweetOrBitter|Rating|Review_Date|
+------------+--------------------+-------------+------+-----------+
|    A. Morin|         Agua Grande|       Bitter|  3.75|       2016|
|    A. Morin|               Kpime|       Bitter|  2.75|       2015|
|    A. Morin|              Atsane|       Bitter|   3.0|       2015|
|    A. Morin|               Akata|       Bitter|   3.5|       2015|
|    A. Morin|              Quilla|       Bitter|   3.5|       2015|
+------------+--------------------+-------------+------+-----------+
only showing top 5 rows



### <font color="magenta">Q5: Show Company_Name, Specific_Bean_Origin, Review_Date, and Descriptive Rating according to the following conditions:

\>=5: Elite (Transcending beyond the ordinary limits)

\>=4: Premium (Superior flavor development, character and style)

\>=3: Satisfactory (well made with special qualities)

\>=2: Disappointing (Passable but contains at least one significant flaw)

<2: Unpleasant (mostly unpalatable)

In [51]:
df.select("Company_Name", 
          "Specific_Bean_Origin",
          "Review_Date",
          F.when(df.Rating >= 5, "Elite")
          .when((df.Rating >= 4) & (df.Rating < 5), "Premium")
          .when((df.Rating >= 3) & (df.Rating < 4), "Satisfactory")
          .when((df.Rating >= 2) & (df.Rating < 3), "Disappointing")
          .otherwise("Unpleasant").alias("Descriptive_Rating")).show(20)

+------------+--------------------+-----------+------------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Descriptive_Rating|
+------------+--------------------+-----------+------------------+
|    A. Morin|         Agua Grande|       2016|      Satisfactory|
|    A. Morin|               Kpime|       2015|     Disappointing|
|    A. Morin|              Atsane|       2015|      Satisfactory|
|    A. Morin|               Akata|       2015|      Satisfactory|
|    A. Morin|              Quilla|       2015|      Satisfactory|
|    A. Morin|            Carenero|       2014|     Disappointing|
|    A. Morin|                Cuba|       2014|      Satisfactory|
|    A. Morin|        Sur del Lago|       2014|      Satisfactory|
|    A. Morin|      Puerto Cabello|       2014|      Satisfactory|
|    A. Morin|             Pablino|       2014|           Premium|
|    A. Morin|              Panama|       2013|     Disappointing|
|    A. Morin|          Madagascar|       2013|      Satisfact

### Select Columns with Arithmetic Manipulation

We can apply arithmetic operation on a column during column selection, for example:

In [52]:
df.select(df["Cocoa_Percent"]/100).show(1)

+---------------------+
|(Cocoa_Percent / 100)|
+---------------------+
|                 0.63|
+---------------------+
only showing top 1 row



### <font color="magenta">Q6: Suppose now we define a cocoa tasty score as the product of cocoa_percentage and rating divided by 100. Compute and show the score.

In [58]:
df.select((df.Cocoa_Percent*df.Rating)/100).show(1)

+--------------------------------+
|((Cocoa_Percent * Rating) / 100)|
+--------------------------------+
|                          2.3625|
+--------------------------------+
only showing top 1 row



#### Summarizing statistics of a variable is similar to Pandas

In [59]:
df.select("Rating").describe().show()

+-------+------------------+
|summary|            Rating|
+-------+------------------+
|  count|              1795|
|   mean| 3.185933147632312|
| stddev|0.4780623935863306|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



## Filter

Remember boolean masking in Pandas? We can do the same thing in PySpark. For example, if we want to see only the ratings reviewed in 2014:

In [60]:
df.filter(df["Review_Date"]==2014).show(5)

+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|    A. Morin|            Carenero|       2014|         70.0|          France|  2.75|  Criollo|        Venezuela|
|    A. Morin|                Cuba|       2014|         70.0|          France|   3.5|         |             Cuba|
|    A. Morin|        Sur del Lago|       2014|         70.0|          France|   3.5|  Criollo|        Venezuela|
|    A. Morin|      Puerto Cabello|       2014|         70.0|          France|  3.75|  Criollo|        Venezuela|
|    A. Morin|             Pablino|       2014|         70.0|          France|   4.0|         |             Peru|
+------------+--------------------+-----------+-------------+----------------+------+---

### <font color="magenta">Q7: Find the records of chocolate bars with ratings being at least 3 (including 3).

In [61]:
df.filter(df["Rating"] >= 3).show(5)

+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|    A. Morin|         Agua Grande|       2016|         63.0|          France|  3.75|         |         Sao Tome|
|    A. Morin|              Atsane|       2015|         70.0|          France|   3.0|         |             Togo|
|    A. Morin|               Akata|       2015|         70.0|          France|   3.5|         |             Togo|
|    A. Morin|              Quilla|       2015|         70.0|          France|   3.5|         |             Peru|
|    A. Morin|                Cuba|       2014|         70.0|          France|   3.5|         |             Cuba|
+------------+--------------------+-----------+-------------+----------------+------+---

Furthermore, if you want to filter by checking the field against multiple values, use .isin() 

In [62]:
df.filter(df["Company_Name"].isin("Chocola'te","Malmo")).show(5)

+------------+--------------------+-----------+-------------+----------------+------+----------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating| Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+----------+-----------------+
|  Chocola'te|          Madagascar|       2011|         70.0|          U.S.A.|  3.75|Trinitario|       Madagascar|
|  Chocola'te|           Venezuela|       2011|         68.0|          U.S.A.|  3.75|          |        Venezuela|
|       Malmo|               Chuao|       2016|         70.0|          Sweden|   3.0|          |        Venezuela|
+------------+--------------------+-----------+-------------+----------------+------+----------+-----------------+



### <font color="magenta">Q8: Show the company names and ratings of chocolate bars produced by a French or Belgian company and reviewed in 2015. 

Hint: remember boolean masking with multiple conditions in Pandas? Similarly, we can use **&** operator here.

In [71]:
df.select('Company_Name','Rating').filter(df['Company_Location'].isin('France','Belgium')).filter(df['Review_Date'] == 2015).show(5)

+-----------------+------+
|     Company_Name|Rating|
+-----------------+------+
|         A. Morin|  2.75|
|         A. Morin|   3.0|
|         A. Morin|   3.5|
|         A. Morin|   3.5|
|Frederic Blondeel|   3.5|
+-----------------+------+
only showing top 5 rows



## Sort



If you want to sort the chocolate bars firstly by cocoa percentage in descending order, and secondly by company in alphabetic order, then you can do:

In [72]:
df.orderBy(["Cocoa_Percent", "Company_Name"], ascending=[False,True]).show(5)

+--------------------+--------------------+-----------+-------------+----------------+------+------------------+-------------------+
|        Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|         Bean_Type|  Broad_Bean_Origin|
+--------------------+--------------------+-----------+-------------+----------------+------+------------------+-------------------+
| Artisan du Chocolat|           Venezuela|       2010|        100.0|            U.K.|  1.75|                  |          Venezuela|
|              Bonnat|         One Hundred|       2006|        100.0|          France|   1.5|                  |                   |
|Bouga Cacao (Tuli...|El Oro, Hacienda ...|       2009|        100.0|         Ecuador|   1.5|Forastero (Arriba)|            Ecuador|
|             C-Amaro|             Ecuador|       2013|        100.0|           Italy|   3.5|                  |            Ecuador|
|     Claudio Corallo|            Principe|       2008|        100.0|

### <font color="magenta">Q9: Kai wants to buy chocolate bars for Valentine's Day. Provide a sorted list for him to review. His preference is listed below in order:

1. Must be reviewed in 2016

2. The broad bean origin must be from Brazil, or Peru

3. The higher the rating, the better

4. The higher the cocoa percent, the better

In [78]:
df.select(['Company_Name','Specific_Bean_Origin','Rating','Cocoa_Percent'])\
    .filter(df.Review_Date==2016).filter(df.Specific_Bean_Origin.isin('Brazil','Peru'))\
    .orderBy(['Rating','Cocoa_Percent'], ascending=[False,False]).show(10)

+--------------------+--------------------+------+-------------+
|        Company_Name|Specific_Bean_Origin|Rating|Cocoa_Percent|
+--------------------+--------------------+------+-------------+
|               Batch|              Brazil|  3.75|         70.0|
|Manufaktura Czeko...|                Peru|   3.5|         70.0|
|                 Cao|              Brazil|  3.25|         70.0|
|          Summerbird|                Peru|   3.0|         71.0|
|          Two Ravens|                Peru|   3.0|         60.0|
|           Heilemann|                Peru|  2.75|         64.0|
|               Vivra|                Peru|   2.5|         70.0|
|          Summerbird|                Peru|   2.5|         61.0|
+--------------------+--------------------+------+-------------+



## Group by and Aggregation

You can summarize statistics in different groups in this way:

In [79]:
df.groupBy("Review_Date").agg({'Rating': 'mean'}).show()

+-----------+------------------+
|Review_Date|       avg(Rating)|
+-----------+------------------+
|       2007|3.1623376623376624|
|       2015|3.2464912280701754|
|       2006|             3.125|
|       2013|3.1970108695652173|
|       2014|3.1892712550607287|
|       2012| 3.178205128205128|
|       2009| 3.073170731707317|
|       2016|3.2260273972602738|
|       2010|3.1486486486486487|
|       2011| 3.256060606060606|
|       2008|2.9946236559139785|
|       2017|            3.3125|
+-----------+------------------+



In [80]:
df.groupBy("Review_Date").agg({'Rating': 'max'}).show()

+-----------+-----------+
|Review_Date|max(Rating)|
+-----------+-----------+
|       2007|        5.0|
|       2015|        4.0|
|       2006|        5.0|
|       2013|        4.0|
|       2014|        4.0|
|       2012|        4.0|
|       2009|        4.0|
|       2016|        4.0|
|       2010|        4.0|
|       2011|        4.0|
|       2008|        4.0|
|       2017|       3.75|
+-----------+-----------+



### <font color="magenta">Q10: Find the average rating for each company for each review year, of the British chocolate bars, sorted by 1) year from the earliest to latest and 2) averag rating from highest to lowest. 

In [85]:
uk_bars = df.filter(df['Company_Location'] == "U.K.")\
                .groupBy(['Company_Name','Review_Date']).agg({'Rating': 'mean'})\
                .orderBy(['Review_Date','avg(Rating)'], ascending=[True,False]).show(20)

+--------------------+-----------+------------------+
|        Company_Name|Review_Date|       avg(Rating)|
+--------------------+-----------+------------------+
|Green & Black's (...|       2006|               2.5|
| Artisan du Chocolat|       2008|              3.75|
|Hotel Chocolat (C...|       2008|            2.8125|
|      Willie's Cacao|       2009|             3.125|
| Artisan du Chocolat|       2009|2.7083333333333335|
|      Willie's Cacao|       2010|3.4166666666666665|
|             Duffy's|       2010|             3.375|
| Artisan du Chocolat|       2010|               3.1|
|Hotel Chocolat (C...|       2010|2.8333333333333335|
|             Duffy's|       2011|               3.4|
| Artisan du Chocolat|       2011|             3.375|
|Hotel Chocolat (C...|       2011|3.0833333333333335|
|             Duffy's|       2012|              3.75|
| Artisan du Chocolat|       2012|              3.75|
|Rococo (Grenada C...|       2012|               3.5|
|Hotel Chocolat (C...|      

## Run SQL Queries in Spark

To run SQL queries, we need to first create a temporary view from a Spark DataFrame. We can do this using .createOrReplaceTempView(), which creates a new view or replace an existing view:

In [98]:
chocolate_bars = df.createOrReplaceTempView("chocolate_bars")

Let's run our first query to show the temporary view we just created:

In [99]:
spark.sql("SELECT * FROM chocolate_bars").show(1)

+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|Company_Name|Specific_Bean_Origin|Review_Date|Cocoa_Percent|Company_Location|Rating|Bean_Type|Broad_Bean_Origin|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
|    A. Morin|         Agua Grande|       2016|         63.0|          France|  3.75|         |         Sao Tome|
+------------+--------------------+-----------+-------------+----------------+------+---------+-----------------+
only showing top 1 row



### <font color="magenta">Q11: Write a Spark SQL query to find the average rating for each company that satisfies the following conditions:

**1. the chocolate bars were reviewed in 2016 **

**2. the chocolate bars were made by companies in this list: 'Germany','France','Spain','Sweden','Denmark','Finland'**

**3. the company should have at least 2 chocolate bar reviews **


In [102]:
spark.sql(
    """
    SELECT 
    Company_Name,
    COUNT(Company_Name) AS Count,
    AVG(Rating) AS Avg_Rating
    FROM chocolate_bars
    WHERE Review_Date = 2016
    AND Company_Location IN ('Germany', 'France', 'Spain', 'Sweden', 'Denmark', 'Finland')
    GROUP BY Company_Name
    HAVING Count >= 2
    """).show()

+------------+-----+------------------+
|Company_Name|Count|        Avg_Rating|
+------------+-----+------------------+
|    Belyzium|    3|3.0833333333333335|
|  Summerbird|    2|              2.75|
+------------+-----+------------------+



## Pandas Interoperability

### Transform Spark DataFrame to Pandas DataFrame

You can convert the Spark DataFrame into a Pandas DataFrame using .toPandas().

### <font color="magenta">Q12: 
### <font color="magenta"> (1) Write a Spark SQL to find the average cocoa percentage for each company in the U.K. and order the result by the average cocoa percentage from highest to lowest. 
### <font color="magenta"> (2) Convert the query result into a Pandas DataFrame, save it into a variable "query_result" , and show it.

In [105]:
query_result = spark.sql(
    """
    SELECT 
    Company_Name,
    AVG(Cocoa_Percent) AS Avg_Cocoa_Percent
    FROM chocolate_bars
    WHERE Company_Location = 'U.K.'
    GROUP BY Company_Name
    ORDER BY Avg_Cocoa_Percent DESC
    """)

In [106]:
query_result.toPandas() 

,Company_Name,Avg_Cocoa_Percent
0,Hotel Chocolat,80.000000
1,Hotel Chocolat (Coppeneur),78.105263
2,Willie's Cacao,76.500000
3,Solkiki,76.500000
4,Pump Street Bakery,76.000000
5,Dormouse,75.500000
6,Doble & Bignall,75.250000
7,Artisan du Chocolat,74.625000
8,Beau Cacao,72.500000
9,Artisan du Chocolat (Casa Luker),72.000000


### Create Pandas DataFrame to Spark DataFrame

You can convert "query_result" back to a Spark DataFrame using spark.createDataFrame()

In [39]:
spark.createDataFrame(query_result).show()

+--------------------+-----------------+
|        Company_Name|AVG_Cocoa_Percent|
+--------------------+-----------------+
|      Hotel Chocolat|             80.0|
|Hotel Chocolat (C...|78.10526315789474|
|      Willie's Cacao|             76.5|
|             Solkiki|             76.5|
|  Pump Street Bakery|             76.0|
|            Dormouse|             75.5|
|     Doble & Bignall|            75.25|
| Artisan du Chocolat|           74.625|
|          Beau Cacao|             72.5|
|Artisan du Chocol...|             72.0|
|       Forever Cacao|             72.0|
|             Duffy's|70.84615384615384|
|            Seaforth|             70.0|
|         Chocolarder|             70.0|
|Black River (A. M...|             70.0|
|           Chocablog|             70.0|
|Green & Black's (...|             70.0|
|              Damson|             70.0|
|          Paul Young|             68.5|
|Rococo (Grenada C...|             66.0|
+--------------------+-----------------+



# End of Lab 5